In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# FactCoreTruncateInsertLoader: order_items_Fact Core TruncateInsertLoader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#core#database_name}`.`{marc_databricks_initiative#core#schema_name}`.`cor_f_order_items` (
         `bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`ent_orders_ent_orders_id`
        ,`ent_customers_ent_customers_id`
        ,`ent_products_ent_products_id`
        ,`ent_stores_ent_stores_id`
        ,`unit_price`
        ,`quantity`
    )
    SELECT
         {bg_loadtimestamp} AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`ent_orders_ent_orders_id` AS `ent_orders_ent_orders_id`
        ,`bg_source`.`ent_customers_ent_customers_id` AS `ent_customers_ent_customers_id`
        ,`bg_source`.`ent_products_ent_products_id` AS `ent_products_ent_products_id`
        ,`bg_source`.`ent_stores_ent_stores_id` AS `ent_stores_ent_stores_id`
        ,`bg_source`.`unit_price` AS `unit_price`
        ,`bg_source`.`quantity` AS `quantity`
    FROM `{marc_databricks_initiative#core#database_name}`.`{marc_databricks_initiative#core#schema_name}`.`cor_f_order_items_lookup` AS `bg_source`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['reloadtarget_{marc_databricks_initiative#core#database_name}_{marc_databricks_initiative#core#schema_name}_cor_f_order_items'] = operation_metrics

except Exception as e:
    info('COR_F_order_items_Loader', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
